In [1]:
import numpy as np
import pandas as pd
from StringIO import StringIO
import re
import os
import pickle
import random


class Cell:
    def __init__(self,v,p,fp,e):
        self.e = e
        self.p = p.copy()
        self.fp = fp.copy()
        self.v = v.copy()

        
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

data_dir = 'data_behler'


In [2]:
def read_fetures(data_dir):
    data_X = []
    data_y = []
    data_names = []
    for num,i in enumerate(os.listdir(data_dir)):
        name = data_dir+'/'+i
        with open(name, 'rb') as f:
            arr = pickle.load(f)
        X = np.empty((len(arr.keys())-1,arr[0].shape[0]),dtype=np.float32)
        for j in range(len(arr.keys())-1):
            X[j] = arr[j]
        data_X.append(X)
        data_y.append(arr[-1])
        data_names.append(int(i[:-len('.pkl')]))
        
        if(num % 10 == 0):
            print '\r %i'%num,
    return data_X,np.array(data_y),np.array(data_names)


In [3]:
bechler_f = read_fetures('data_behler')
save_obj(bechler_f,'bechler_f')

 8180                                                                                                                                                                                                          


In [38]:
#cheb_f = read_fetures('data_cheb')
#save_obj(cheb_f,'cheb_f')

 8180                                                                                                                                                                                                                                                                                                               


In [4]:
import theano

Using gpu device 0: GeForce GT 740M (CNMeM is disabled, CuDNN 5105)
/home/nvvaulin/.local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)


In [5]:
import theano.tensor as T
import lasagne

In [6]:
from lasagne.layers import *
from broadcast import *
from lasagne.nonlinearities import *

In [7]:
def train_data_generator(train_data,batch_size,batches_per_epoch):
    data_X = train_data[0]
    data_y = train_data[1]
    natoms = []
    unique,natoms,counts = np.unique(np.array([len(i) for i in data_X]),return_inverse=True,return_counts=True)
    unique_inx = np.arange(len(unique),dtype=np.int)
    natom_inx = []  
    tmp = np.arange(len(natoms),dtype=np.int)
    for i in unique_inx:
        natom_inx.append(tmp[natoms==i])
    w = counts.astype(np.float32)
    w /= w.sum()
    for batch_num in xrange(batches_per_epoch):
        count = np.random.choice(unique_inx,p=w)
        inx = np.random.choice(natom_inx[count],batch_size)        
        batch_y = data_y[inx].copy()
        batch_X = np.empty((batch_size,data_X[0].shape[1],unique[count]),dtype=np.float32)
        for j,i in enumerate(inx):
            batch_X[j] = data_X[i].T.copy()
            
        batch_y = (batch_y - energy_mean)/energy_var
        yield batch_X,batch_y.reshape(-1,1)
        
def val_data_generator(data,batch_size):
    data_X = data[0]
    data_y = data[1]
    lens = np.array([len(i) for i in data_X])
    inxs = np.argsort(lens)
    lens = lens[inxs]
    lo = 0
    while(lo < len(inxs)):
        natoms = lens[lo]
        hi = min(lo+batch_size,np.searchsorted(lens[lo:],natoms,'right')+lo)
        batch_y = data_y[inxs[lo:hi]].copy()
        batch_X = np.empty((hi-lo,data_X[0].shape[1],natoms),dtype=np.float32)
        for j,i in enumerate(inxs[lo:hi]):
            batch_X[j] = data_X[i].T.copy()
        lo = hi
        batch_y = (batch_y - energy_mean)/energy_var
        yield batch_X,batch_y.reshape(-1,1)

In [8]:
def train_test_split(data,val_ratio,seed = 0):
    inx = np.arange(len(data[0]),dtype=int)
    np.random.seed(seed)
    np.random.shuffle(inx)
    sl = int(val_ratio*len(data[0]))
    train_X = []
    val_X = []
    for i in range(sl):
        val_X.append(data[0][inx[i]])
    for i in range(sl,len(data[0])):
        train_X.append(data[0][inx[i]])
    return (train_X,data[1][inx[sl:]]),(val_X,data[1][inx[:sl]])


In [13]:
data_features = bechler_f
energy_mean = data_features[1].mean()
energy_var = np.sqrt(((data_features[1]-energy_mean)**2).mean())
input_shape = (None,data_features[0][0].shape[1],None)
train_data,val_data = train_test_split(data_features,0.1)

In [56]:
def build1(t_input,input_shape = input_shape):#batch,fetures,atoms
    def make_dense(l,n,name,nonl = rectify):
        res = BatchNormLayer(l,name = 'bn_'+name)
        res = DenseLayer(res,n,name = 'fc_'+name,nonlinearity=nonl)
        return res
    
    input_l = InputLayer(input_shape,input_var = t_input)
    print input_l.output_shape
    bcast_l = BroadcastLayer(input_l,[0,2])
    print bcast_l.output_shape
    dense = make_dense(bcast_l,512,'1')
    dense = make_dense(dense,1024,'3',nonl=rectify)
    print dense.output_shape
    unbcast = UnbroadcastLayer(dense,bcast_l)
    print unbcast.output_shape
    glpool = GlobalPoolLayer(unbcast,T.sum)
    print glpool.output_shape
    dense = make_dense(glpool,512,'1')
    dense = make_dense(dense,256,'3',nonl=rectify)

    res = DenseLayer(dense,1,nonlinearity=identity)
    return res

def build(t_input,input_shape = input_shape):#batch,fetures,atoms
    def make_dense(l,n,name,nonl = rectify):
        res = BatchNormLayer(l,name = 'bn_'+name)
        res = DenseLayer(res,n,name = 'fc_'+name,nonlinearity=nonl)
        return res
    
    input_l = InputLayer(input_shape,input_var = t_input)
    print input_l.output_shape
    bcast_l = BroadcastLayer(input_l,[0,2])
    print bcast_l.output_shape
    dense = make_dense(bcast_l,512,'1')
    dense = make_dense(dense,1024,'3',nonl=rectify)
    print dense.output_shape
    unbcast = UnbroadcastLayer(dense,bcast_l)
    print unbcast.output_shape
    glpool = GlobalPoolLayer(unbcast,T.sum)
    print glpool.output_shape
    dense = make_dense(glpool,512,'1')
    dense = make_dense(dense,256,'3',nonl=rectify)
    res = DenseLayer(dense,1,nonlinearity=identity)
    return res


fatures = T.tensor3('fetures')
energies = T.matrix('energy')
net = build(fatures)


(None, 462, None)
(None, 462)
(None, 1024)
(None, 1024, None)
(None, 1024)


In [57]:
pred = get_output(net)
params = get_all_params(net, trainable=True)
loss = lasagne.objectives.squared_error(pred,energies).mean()
G_lr = theano.shared(np.array(0.001, dtype=theano.config.floatX))
updates = lasagne.updates.adam(loss,params,G_lr)
print ('start')
train_fn = theano.function([fatures,energies],[loss], allow_input_downcast=True, updates=updates)
print ('train_fn compiled')
test_fn = theano.function([fatures,energies],[loss], allow_input_downcast=True)
print ('test_fn compiled')

start
train_fn compiled
test_fn compiled


In [58]:
def save_weights(network, name ):
    np.savez(name+".npz", **{"param%d" % i: param for i, param in enumerate(get_all_param_values(network))})
             
def load_weights(network,name ):
    f = np.load(name+".npz")
    params = [f["param%d" % i] for i in range(len(f.files))]
    f.close()
    set_all_param_values(network,params)

def train(num_epoch,train_data_genertor,val_data_genertor,
          train_fn=train_fn,
          test_fn=test_fn,
          net=net,
          model_prefix='model',
          G_lr = G_lr,
          lr_sh = [100,150]):
    
    log = open('train.log','a')
    
    for epoch in range(num_epoch):
        train_num_batch = 0.
        train_loss = 0.
        for i,batch in enumerate(train_data_genertor()):
            train_loss += train_fn(batch[0],batch[1])[0]
            train_num_batch += 1.
            if(i % 10 == 0):
                print '\r %i %f'%(i,train_loss/train_num_batch),
                log.write('%i %f\n'%(i,train_loss/train_num_batch))
        val_loss = 0.
        val_num_batch = 0.
        for i,batch in enumerate(val_data_genertor()):
            val_loss += test_fn(batch[0],batch[1])[0]
            val_num_batch += 1.
        save_weights(net,model_prefix+'repoch%itr%.3fval%.3f'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        print ('\repoch %i train_loss=%f val_loss=%f'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        log.write('epoch %i train_loss=%f val_loss=%f\n'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        
        if(epoch in lr_sh):
            G_lr.set_value(G_lr.get_value()*np.float(0.01))
        
    log.close()
    
    

In [61]:
train_generator = lambda : train_data_generator(train_data,500,200)
val_generator = lambda : val_data_generator(val_data,500)
G_lr.set_value(np.float(0.001))
train(250,train_generator,val_generator)

epoch 0 train_loss=1.085759 val_loss=1.210062
epoch 1 train_loss=0.819076 val_loss=1.219164
epoch 2 train_loss=1.053766 val_loss=1.195876
epoch 3 train_loss=1.053764 val_loss=1.181652
epoch 4 train_loss=1.052548 val_loss=1.162484
epoch 5 train_loss=0.916806 val_loss=1.145805
epoch 6 train_loss=0.728708 val_loss=1.143874
epoch 7 train_loss=0.937789 val_loss=1.063739
epoch 8 train_loss=0.694329 val_loss=1.020797
epoch 9 train_loss=0.806675 val_loss=0.944785
epoch 10 train_loss=0.786716 val_loss=0.804985
epoch 11 train_loss=0.547948 val_loss=0.701317


KeyboardInterrupt: 